## This notebook does the whole-brain voxel-wise GLM fitting for PPI

We use FSL FEAT

In [1]:
import pandas as pd
import numpy as np
import glob
import re

# for masking
import nilearn
from nilearn.input_data import NiftiMasker
import nibabel as nib
import itertools
import multiprocessing as mp

import os
import shutil

# 1. First, we need to highpass the functional data (in both T1w and MNI space)
In the whole-brain GLM, we used cosines to model drift. However, for a PPI, this would create strong multicollinearity with the STN timeseries in the design matrix

In [49]:
def high_pass(nii, verbose=False, mask=None):
    print('Highpass-filtering')
    t_r = nii.header['pixdim'][4]  
    if t_r > 1e2:
        t_r /= 1e3 # new version of nibabel loads t_r in ms...
        
    if mask is not None:
        hp_masker = NiftiMasker(mask, high_pass=1./128, t_r=t_r)
    else:
        hp_masker = NiftiMasker(high_pass=1./128, t_r=t_r)
    
    # Generate & fit NiftiMasker
    hp_data = hp_masker.fit_transform(nii)
    
    # back to brain space
    inver = hp_masker.inverse_transform(hp_data)

    # add mean of timeseries per voxel back
    highpassed_data = inver.get_fdata() + np.mean(nii.get_fdata(), 3)[:,:,:,np.newaxis]
    highpassed_img = nib.Nifti1Image(highpassed_data, inver.affine, header=nii.header)
    
    return highpassed_img

def high_pass_and_save(fn):
    print(fn)
    save_fn = fn.replace('fmriprep', 'high_passed_func')
    save_dir = os.path.dirname(save_fn)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    high_passed = high_pass(nib.load(fn))
    high_passed.to_filename(save_fn)

In [50]:
all_func_files = sorted(glob.glob('./derivatives/ds-0*/fmriprep/sub-*/func/sub-*_task-randomdotmotion_run-*_space-*_desc-preproc_bold.nii.gz'))

with mp.Pool(10) as p:
    p.map(high_pass_and_save, all_func_files)

./derivatives/ds-01/fmriprep/sub-01/func/sub-01_task-randomdotmotion_run-03_space-T1w_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-01/func/sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-02/func/sub-02_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-07/func/sub-07_task-randomdotmotion_run-01_space-T1w_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-03/func/sub-03_task-randomdotmotion_run-02_space-T1w_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-08/func/sub-08_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-05/func/sub-05_task-randomdotmotion_run-01_space-T1w_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-04/func/sub-04_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/fmriprep/sub-06/func/sub-06_task-ran

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obt

./derivatives/ds-01/fmriprep/sub-09/func/sub-09_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-01/fmriprep/sub-07/func/sub-07_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering


/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


./derivatives/ds-01/fmriprep/sub-02/func/sub-02_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-01/fmriprep/sub-05/func/sub-05_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-01/fmriprep/sub-03/func/sub-03_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-01/fmriprep/sub-01/func/sub-01_task-randomdotmotion_run-01_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering


...

./derivatives/ds-02/fmriprep/sub-05/func/sub-05_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-06/func/sub-06_task-randomdotmotion_run-02_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-09/func/sub-09_task-randomdotmotion_run-02_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-10/func/sub-10_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-08/func/sub-08_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-11/func/sub-11_task-randomdotmotion_run-01_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering


/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


./derivatives/ds-02/fmriprep/sub-12/func/sub-12_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering


/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


./derivatives/ds-02/fmriprep/sub-09/func/sub-09_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-07/func/sub-07_task-randomdotmotion_run-01_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-08/func/sub-08_task-randomdotmotion_run-03_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-12/func/sub-12_task-randomdotmotion_run-03_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-13/func/sub-13_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-11/func/sub-11_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-14/func/sub-14_task-randomdotmotion_run-02_space-T1w_desc-preproc_bold.nii.gz
Highpass-filtering
./derivatives/ds-02/fmriprep/sub-13/func/s

## 2. Re-extract STN segments from high-passed func timeseries in T1w space


In [51]:
def get_maskfn_label(ds, sub, hemisphere, subregion):
    if subregion == '':
        label = 'stn{}'.format(hemisphere)
    else:
        label = 'stn{}_{}'.format(hemisphere, subregion)
    subregion = '_' + subregion if not subregion == '' else subregion
    return label, f'./derivatives/ds-{ds}/pca_masks/sub-{sub}/anat/sub-{sub}_desc-stn{hemisphere}_space-T1w_res-1p5mm{subregion}_roi.nii.gz'

def make_subject_atlas(ds, sub):
    hemispheres = ['r', 'l']
    subregions = ['', 'subroi-A', 'subroi-B', 'subroi-C']

    labels = []
    imgs = []
    for hemisphere in hemispheres:
        for subregion in subregions:
            label, fn = get_maskfn_label(ds, sub, hemisphere, subregion)
            labels.append(label)
            imgs.append(fn)

    from sklearn.utils import Bunch
    atlas = Bunch(labels=labels, 
                  maps=nilearn.image.concat_imgs(imgs))
    return atlas

def get_epi(ds, sub, run):
    epi = f'./derivatives/ds-{ds}/high_passed_func/sub-{sub}/func/sub-{sub}_task-randomdotmotion_run-{run}_space-T1w_desc-preproc_bold.nii.gz'
    return epi

def do_extract(to_run):
    ds, sub, run = to_run
    ds = str(ds).zfill(2)
    sub = str(sub).zfill(2)
    run = str(run).zfill(2)
    print(f'Extracting from ds-{ds}/sub-{sub}/run-{run}', end='')

    epi_fn = get_epi(ds, sub, run)
    if not os.path.exists(epi_fn):
        print('...doesnt exist, skipping'.format(ds, sub, run))
        return 0
    
    atlas = make_subject_atlas(ds, sub)
    epi = nib.load(get_epi(ds, sub, run))
    epi_flat = epi.get_fdata().reshape((np.product(epi.shape[:3]), epi.shape[-1]))

    for i in np.arange(len(atlas.labels)):
        print('.', end='')
        label = atlas.labels[i]
        mask = nilearn.image.index_img(atlas.maps, i)
        mask_flat = mask.get_fdata().ravel()
        signal = pd.DataFrame(epi_flat[mask_flat>0].T)
        signal.index.name = 'volume'

        output_fn = f'./derivatives/ds-{ds}/extracted_signal_hp/sub-{sub}/func/sub-{sub}_task-randomdotmotion_run-{run}_space-T1w_desc-preproc_desc-{label}.csv'
        output_fn

        os.makedirs(os.path.dirname(output_fn), exist_ok=True)
        signal.to_csv(output_fn)

In [52]:
all_combinations = list(itertools.product([1,2], np.arange(1,20), [1,2,3]))

with mp.Pool(30) as p:
    p.map(do_extract, all_combinations)

Extracting from ds-01/sub-01/run-01Extracting from ds-01/sub-01/run-02Extracting from ds-01/sub-04/run-03Extracting from ds-01/sub-02/run-02Extracting from ds-01/sub-03/run-03Extracting from ds-01/sub-06/run-02Extracting from ds-01/sub-05/run-02Extracting from ds-01/sub-04/run-01Extracting from ds-01/sub-03/run-02Extracting from ds-01/sub-02/run-01Extracting from ds-01/sub-06/run-01Extracting from ds-01/sub-04/run-02Extracting from ds-01/sub-06/run-03Extracting from ds-01/sub-01/run-03Extracting from ds-01/sub-05/run-01Extracting from ds-01/sub-05/run-03Extracting from ds-01/sub-02/run-03Extracting from ds-01/sub-07/run-01Extracting from ds-01/sub-07/run-02Extracting from ds-01/sub-03/run-01Extracting from ds-01/sub-07/run-03...doesnt exist, skippingExtracting from ds-01/sub-08/run-01Extracting from ds-01/sub-09/run-01Extracting from ds-01/sub-08/run-03Extracting from ds-01/sub-09/run-02Extracting from ds-01/sub-08/run-02
Extracting from ds-01/sub-10/run-01Extracting from ds-01/sub-09/

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obt

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obt

.

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


...

.......Extracting from ds-02/sub-14/run-02.

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


.......Extracting from ds-02/sub-14/run-03

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obt

........Extracting from ds-02/sub-15/run-01........Extracting from ds-02/sub-15/run-02........Extracting from ds-02/sub-15/run-03........Extracting from ds-02/sub-16/run-01...doesnt exist, skipping
Extracting from ds-02/sub-16/run-02...doesnt exist, skipping
Extracting from ds-02/sub-16/run-03...doesnt exist, skipping
Extracting from ds-02/sub-17/run-01...doesnt exist, skipping
Extracting from ds-02/sub-17/run-02...doesnt exist, skipping
Extracting from ds-02/sub-17/run-03...doesnt exist, skipping
Extracting from ds-02/sub-18/run-01...doesnt exist, skipping
Extracting from ds-02/sub-18/run-02...doesnt exist, skipping
Extracting from ds-02/sub-18/run-03...doesnt exist, skipping
Extracting from ds-02/sub-19/run-01...doesnt exist, skipping
Extracting from ds-02/sub-19/run-02...doesnt exist, skipping
Extracting from ds-02/sub-19/run-03...doesnt exist, skipping
...................................................................................................................................

In [75]:
all_csvs = sorted(glob.glob('./derivatives/ds-*/extracted_signal_hp/sub-*/func/*T1w*subroi*.csv'))
reg = re.compile('.*ds-(?P<ds>\d+)/extracted_signal_hp/sub-(?P<sub>\d+)/func/sub-.*_task-randomdotmotion_run-(?P<run>\d+)_space-T1w_desc-preproc_desc-stn(?P<hemisphere>\S+)_subroi-(?P<roi>\S).*')

df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    d['signal'] = pd.read_csv(fn, index_col=0).mean(1)
    d = pd.DataFrame(d)
    d['t'] = np.arange(0, d.shape[0]*3, 3)
    
    df.append(d)

df = pd.concat(df, axis=0)
df['ds'] = df['ds'].astype(int)
df['run'] = df['run'].astype(int)
df['sub'] = df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = df.loc[df.ds==1,'sub'].max()
df.loc[df.ds==2,'sub'] = df.loc[df.ds==2,'sub']+max_sub_ds1

df_wide = df.rename(columns={'sub':'subject'}).pivot_table(values=['signal'], index=['ds', 'subject', 'run', 't'], columns=['hemisphere', 'roi'])
df_wide = df_wide.reset_index(level=0, drop=True)
df_wide.columns = ['_'.join(col) for col in df_wide.columns]
stn_df = df_wide

display(stn_df.head())

signal_l_A   signal_l_B   signal_l_C   signal_r_A  \
subject run t                                                        
1       1   0   1406.334345  1466.586644  1329.272934  1530.516673   
            3   1400.853113  1501.684212  1344.501035  1533.905607   
            6   1408.809741  1494.806150  1339.771262  1544.396568   
            9   1414.084690  1501.861026  1392.506021  1551.420010   
            12  1413.863673  1490.686385  1364.834639  1549.337535   

                 signal_r_B   signal_r_C  
subject run t                             
1       1   0   1546.514125  1521.955931  
            3   1560.962350  1541.748042  
            6   1564.006076  1528.674861  
            9   1570.826043  1515.060187  
            12  1584.529123  1543.239910

# 3. Get events, confounds

In [86]:
events = pd.read_pickle('./derivatives/all_subjects_events.pkl')
confounds = pd.read_pickle('./derivatives/all_subjects_confounds.pkl')[['dvars', 'framewise_displacement', 'rot_x', 'rot_y',
       'rot_z', 'trans_x', 'trans_y', 'trans_z']]
events['onset'] -= 1.5  # slice time correction

In [87]:
def get_sub_n(sub, max_sub_ds1=19):
    if sub > max_sub_ds1:
        sub -= max_sub_ds1
        ds = 2
    else:
        ds = 1
    return ds, sub

#### Events of interest
Unlike the first GLM, we only model 'cue payoff' (not left / right)

In [89]:
include_events = ['easy', 'hard', 'error', 'response_left', 'response_right', 'cue_payoff', 'cue_neutral']
events.loc[events.event_type.isin(['cue_left', 'cue_right']), 'event_type'] = 'cue_payoff'

for (subject, run), events_ in events.groupby(level=[0,1]):
    ds, sub = get_sub_n(subject)
    output_dir = './derivatives/ds-{ds}/feat_files_ppi/sub-{sub}/run-{run}/evs/'.format(**{'ds':str(ds).zfill(2), 'sub':str(sub).zfill(2), 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    events_['weight'] = 1
    for i, ev in enumerate(include_events):
        events_.loc[events_['event_type']==ev, ['onset', 'duration', 'weight']].to_csv(output_dir+'{}.txt'.format(ev), sep='\t', index=False, header=False)
        
for (subject, run), data in stn_df.groupby(level=[0,1]):
    ds, sub = get_sub_n(subject)
    output_dir = './derivatives/ds-{ds}/feat_files_ppi/sub-{sub}/run-{run}/evs/'.format(**{'ds':str(ds).zfill(2), 'sub':str(sub).zfill(2), 'run':run})
    
    for segment in stn_df.columns:
        data[segment].to_csv(output_dir+'{}.txt'.format(segment), sep='\t', index=False, header=False)
        #data['signal_r'].to_csv(output_dir+'{}.txt'.format('stn_r'), sep='\t', index=False, header=False)

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


#### Confounds

In [90]:
for (subject, run), confounds_ in confounds.groupby(level=[0,1]):
    ds, sub = get_sub_n(subject)
    output_dir = './derivatives/ds-{ds}/feat_files_ppi/sub-{sub}/run-{run}/evs/'.format(**{'ds':str(ds).zfill(2), 'sub':str(sub).zfill(2), 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    confounds_.to_csv(output_dir+'confounds.txt', sep='\t', index=False, header=False)

## mask & fix header
Unfortunately the header says the TR is 3000, which FSL interprets as seconds. We need to fix

In [99]:
def mask_img(img, mask):
    masker = NiftiMasker(mask)
    data_masked = masker.fit_transform(img)
    data_masked_epispace = masker.inverse_transform(data_masked)
    return data_masked_epispace

def mask_data_and_fix_header(arg):
    sub, run = arg
    if sub == 23:  # does not exist
        return 0
    if sub == 6 and run == 3: # does not exist
        return 0
    if sub == 26 and run == 3: # does not exist
        return 0
    
    ds, sub = get_sub_n(sub)
    input_dir = './derivatives/ds-{}/high_passed_func/sub-{}/func'.format(str(ds).zfill(2), str(sub).zfill(2))
    output_dir = './derivatives/ds-{}/high_passed_func/sub-{}/func'.format(str(ds).zfill(2), str(sub).zfill(2))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    img_fn = 'sub-{}_task-randomdotmotion_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(str(sub).zfill(2), str(run).zfill(2))
    input_fn = os.path.join(input_dir, img_fn)
    mask_fn = input_fn.replace('desc-preproc_bold', 'desc-brain_mask').replace('high_passed_func', 'fmriprep')
    output_fn = os.path.join(output_dir, img_fn)
    
#     if os.path.exists(output_fn):
#         return 0
    
    print(input_fn)
    masked_data = mask_img(input_fn, mask_fn)
    nib.save(masked_data, output_fn)
    
    # make sure header is OK
    import subprocess
    return_code = subprocess.run(["fslmerge", "-tr", output_fn, output_fn, "3"])

    return 0

subjects = np.arange(1, 35)
runs = [1,2,3]

to_run = itertools.product(subjects, runs)
to_run_list = list(to_run)
# the following runs don't exist
to_run_list.pop(to_run_list.index((6,3)))
to_run_list.pop(to_run_list.index((26,3)))
to_run_list.pop(to_run_list.index((23,1)))
to_run_list.pop(to_run_list.index((23,2)))
to_run_list.pop(to_run_list.index((23,3)))

# # sequential:
# for i in to_run_list:
#     print(i)
#     mask_data_and_fix_header(i)

# MP
with mp.Pool(10) as p:
    p.map(mask_data_and_fix_header, to_run_list)

./derivatives/ds-01/high_passed_func/sub-05/func/sub-05_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-07/func/sub-07_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-03/func/sub-03_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-06/func/sub-06_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-08/func/sub-08_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-04/func/sub-04_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz./derivatives/ds-01/high_passed_func/sub-09/func/sub-09_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/high_passed_func/sub-01/func/sub-01_task-

# 4. Generate fsf-files for each subject
Note that we first manually create a fsf-file using the FSL FEAT GUI, and then simply change all the paths within the generated fsf-file. We set a smoothing kernel of FWHM=5mm, disable all other preprocessing. 21 contrasts were defined:

- 10 EVs:
    1. stim_easy
    2. stim_hard
    3. cue_payoff
    4. cue_neutral
    5. response_left
    6. response_right
    7. error
    8. STN segment (e.g., left A)
    9. STN \*payoff
    10. STN \*neutral
- The actual constrast of interest:
    11. (STN \* Payoff) - (STN \* Neutral)
    
(I should really learn how to use nipype for this).

We repeat this GLM for:
3 segments * 2 hemispheres * 2 manipulations (hard/easy) = 12 GLMs in total

In [115]:
import copy
# function to adapt a design file
def fix_fsf_file(sub, run,
                 write_out=True):
    
    ds, sub = get_sub_n(sub)
    
    for domain in ['limbic', 'cognitive']:
        # read fsf as txt
        orig_fsf_fn = './derivatives/ds-01/feat_files_ppi/sub-01/{}_design.fsf'.format(domain)
        with open(orig_fsf_fn, 'r') as f:
            orig_fsf = f.read()

        for hemisphere in ['left', 'right']:
            for segment in ['A', 'B', 'C']:
                fsf = copy.copy(orig_fsf)
                # replace output directory
#                 "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/cognitive/left/A/firstlevel"
#                 "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/cognitive/left/A/firstlevel"'
                fsf = fsf.replace('set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/{}/left/A/firstlevel"'.format(domain),
                                  'set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}/{}/{}/{}/firstlevel"'.format(
                                      str(ds).zfill(2), 
                                      str(sub).zfill(2), 
                                      run, domain,
                                      hemisphere,
                                      segment))
                print('set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}/{}/{}/{}/firstlevel"'.format(
                                      str(ds).zfill(2), 
                                      str(sub).zfill(2), 
                                      run, domain,
                                      hemisphere,
                                      segment))
                # input file
                fsf = fsf.replace('/home/stevenm/Projects/bias/derivatives/ds-01/high_passed_func/sub-01/func/sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold',
                                  '/home/stevenm/Projects/bias/derivatives/ds-{}/high_passed_func/sub-{}/func/sub-{}_task-randomdotmotion_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold'.format(str(ds).zfill(2), str(sub).zfill(2), str(sub).zfill(2), str(run).zfill(2)))

                # replace directories for func files & ev files
                fsf = fsf.replace("/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1",
                                  "/home/stevenm/Projects/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}".format(str(ds).zfill(2), str(sub).zfill(2), run))
                
                # replace ev name of signal_l_A
                fsf = fsf.replace('signal_l_A.txt', 'signal_{}_{}.txt'.format(hemisphere[0], segment))

                # save fsf as txt
                if write_out:
                    output_dir = "/home/stevenm/Projects/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}/{}/{}/{}/".format(str(ds).zfill(2), str(sub).zfill(2), run, domain, hemisphere, segment)
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
                        f.write(fsf)
                else:
                    return fsf

    
subjects = np.arange(1, 35)
runs = [1,2,3]

for sub in subjects:
    if sub == 23:
        continue
    if sub == 6 or sub == 26:
        runs = [1,2]
    else:
        runs = [1,2,3]
    for run in runs:
        fix_fsf_file(sub, run)

set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/left/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/left/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/left/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/right/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/right/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/limbic/right/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/cognitive/left/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-1/cognitive/left/B/firstlevel"
set fmri(outputdir) "/h

set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/limbic/right/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/limbic/right/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/left/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/left/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/left/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/right/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/right/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-01/feat_files_ppi/sub-14/run-3/cognitive/right/C/firstlevel"
set fmri(

set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-2/cognitive/right/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/left/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/left/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/left/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/right/A/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/right/B/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/limbic/right/C/firstlevel"
set fmri(outputdir) "/home/stevenm/Projects/bias/derivatives/ds-02/feat_files_ppi/sub-10/run-3/cognitive/left/A/firstlevel"
set fmri(outputdir) "/

# 4. Run first-level FEAT

In [9]:
all_fsfs = sorted(glob.glob('./derivatives/ds-*/feat_files_ppi/sub-*/run*/*/*/*/design.fsf'))
len(all_fsfs)

1164

This will take a while...
- In total, there's 97 runs (33 subs x 3 - 2), with 12 GLMs each
- 15 cpus in parallel?
- Takes about 1.5h per GLM -> 1746 hours / 15cpus = 4.85 days
- Or with 20 cpus 3.6 days (we're nearing the maximum RAM usage with 20 cpus)


In [ ]:
def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/ds-*/feat_files_ppi/sub-*/run*/*/*/*/design.fsf'))
all_fsfs

with mp.Pool(15) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/ds-01/feat_files_ppi/sub-01/run-2/limbic/left/C/design.fsf./derivatives/ds-01/feat_files_ppi/sub-02/run-3/cognitive/left/A/design.fsf./derivatives/ds-01/feat_files_ppi/sub-01/run-1/cognitive/left/A/design.fsf./derivatives/ds-01/feat_files_ppi/sub-03/run-1/limbic/left/C/design.fsf./derivatives/ds-01/feat_files_ppi/sub-05/run-2/cognitive/right/B/design.fsf

./derivatives/ds-01/feat_files_ppi/sub-08/run-1/cognitive/left/A/design.fsf./derivatives/ds-01/feat_files_ppi/sub-04/run-2/cognitive/left/A/design.fsf./derivatives/ds-01/feat_files_ppi/sub-07/run-2/cognitive/right/B/design.fsf./derivatives/ds-01/feat_files_ppi/sub-06/run-2/limbic/left/C/design.fsf./derivatives/ds-01/feat_files_ppi/sub-08/run-2/limbic/left/C/design.fsf./derivatives/ds-01/feat_files_ppi/sub-06/run-1/cognitive/left/A/design.fsf./derivatives/ds-01/feat_files_ppi/sub-02/run-1/cognitive/right/B/design.fsf./derivatives/ds-01/feat_files_ppi/sub-03/run-3/cognitive/right/B/design.fsf./derivatives/ds-01/feat_files_

## 5. Second-level GLM

In the second level, we combine runs per subject using FEAT OLS.

#### Interim: Fix "registration not run"-issue for fixed effects

Feat higher-order analyses require "registration to have been performed", which is true, but not by FSL so FSL doesn't know. To do this, "trick" FSL into thinking they have been run by linking identity matrices to the feat dirs.

See also https://neurostars.org/t/performing-full-glm-analysis-with-fsl-on-the-bold-images-preprocessed-by-fmriprep-without-re-registering-the-data-to-the-mni-space/784


In [36]:
def link_files(feat_dir, fsl_dir='/usr/share/fsl/5.0'):
    reg_dir = os.path.join(feat_dir, 'reg')
    if not os.path.exists(reg_dir):
        os.makedirs(reg_dir)
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/example_func2standard.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/standard2example_func.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/mean_func.nii.gz {}/reg/standard.nii.gz'.format(feat_dir, feat_dir))
    return 0

subjects = np.arange(1, 35)
runs = [1,2,3]

for sub in subjects:
    if sub == 23:
        continue
    if sub == 6 and run == 26:
        runs = [1,2]
    else:
        runs = [1,2,3]
    
    ds, sub = get_sub_n(sub)
    for run in runs:
        feat_dir = '/home/stevenm/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}/firstlevel.feat'.format(str(ds).zfill(2), str(sub).zfill(2), run)
        if os.path.exists(feat_dir):
            link_files(feat_dir)


# 6. Generate .fsf-files for second-level fixed-effects


In [37]:
def fix_fsf_file_fe(sub, orig_fsf='./derivatives/ds-01/feat_files_ppi/sub-01/secondlevel/design.fsf', 
                    write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    orig_sub = sub
    ds, sub = get_sub_n(sub)
    for run in [1,2,3]:
        # replace directories..
        fsf = fsf.replace("/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-01/run-{}/firstlevel.feat".format(run),
                          "/home/stevenm/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-{}/firstlevel.feat".format(str(ds).zfill(2), str(sub).zfill(2), run))
    
    # output directory
    fsf = fsf.replace('set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-01/secondlevel"',
                      'set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/secondlevel"'.format(str(ds).zfill(2), str(sub).zfill(2)))
    
    # mask
    fsf = fsf.replace('set fmri(threshmask) "/home/stevenm/bias/derivatives/ds-01/fmriprep/sub-01/func/sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"',
                      'set fmri(threshmask) "/home/stevenm/bias/derivatives/ds-{}/fmriprep/sub-{}/func/sub-{}_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"'.format(str(ds).zfill(2), str(sub).zfill(2), str(sub).zfill(2)))
    
    # For subject 6 and 26, remove run 3
    if orig_sub == 6 or orig_sub == 26:
        # change n inputs
        fsf = fsf.replace("set fmri(npts) 3", "set fmri(npts) 2")
        fsf = fsf.replace("set fmri(multiple) 3", "set fmri(multiple) 2")
        
        # change feat directories
        fsf = fsf.replace('\n# 4D AVW data or FEAT directory (3)', '')
        fsf = fsf.replace('\nset feat_files(3) "/home/stevenm/bias/derivatives/ds-{}/feat_files_ppi/sub-{}/run-3/firstlevel.feat"'.format(str(ds).zfill(2), str(sub).zfill(2)), '')
        
        # change EV specification
        fsf = fsf.replace('\n# Higher-level EV value for EV 1 and input 3', '')
        fsf = fsf.replace('\nset fmri(evg3.1) 1.0', '')
        fsf = fsf.replace('\n# Group membership for input 3', '')
        fsf = fsf.replace('\nset fmri(groupmem.3) 1', '')
        
    # save fsf as txt
    if write_out:
        output_dir = './derivatives/ds-{}/feat_files_ppi/sub-{}/secondlevel'.format(str(ds).zfill(2), str(sub).zfill(2))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
#fix_fsf_file_fe('01', 'me', write_out=False)

subjects = np.arange(1, 35)
                      
for sub in subjects:
    if sub == 23:
        continue
    else:
        fix_fsf_file_fe(sub, write_out=True)

In [41]:
## run again

def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/ds-*/feat_files_ppi/sub-*/secondlevel/design.fsf'))
#all_fsfs.pop(1)
all_fsfs
with mp.Pool(30) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/ds-01/feat_files_ppi/sub-06/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-13/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-08/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-09/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-02/secondlevel/design.fsf
./derivatives/ds-02/feat_files_ppi/sub-01/secondlevel/design.fsf
./derivatives/ds-02/feat_files_ppi/sub-06/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-18/secondlevel/design.fsf
./derivatives/ds-02/feat_files_ppi/sub-09/secondlevel/design.fsf
./derivatives/ds-02/feat_files_ppi/sub-03/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-16/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-14/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-11/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-17/secondlevel/design.fsf
./derivatives/ds-01/feat_files_ppi/sub-03/secondlevel/design.fsf
./derivatives/ds-01/feat_

## third level made manually

In [42]:
import glob

ls = sorted(glob.glob('/home/stevenm/bias/derivatives/ds-*/feat_files_ppi/sub*/secondlevel.gfeat/cope1.feat'))
print('\n'.join(ls))

/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-01/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-02/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-03/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-04/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-05/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-06/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-07/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-08/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-09/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-10/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files_ppi/sub-11/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/d

In [43]:
# The third-level GLM has four EVs (and an intercept):
import pandas as pd 
all_parameters = pd.read_pickle('./derivatives/all_subjectwise_parameters.pkl')
all_parameters = all_parameters[['ddm z_cue_regressor', 'ddm difficulty_effect']]
# all_parameters

design_matrix = pd.DataFrame([1] * all_parameters.shape[0], columns=['intercept'])
design_matrix['dataset'] = [-1] * 19 + [1] * 14
#design_matrix['dataset_leipzig'] = [1] * 19 + [0] * 14
# design_matrix['z_shift'] = all_parameters['ddm z_cue_regressor'].values
# design_matrix['v_shift'] = all_parameters['ddm difficulty_effect'].values
# design_matrix['z_shift'] -= design_matrix['z_shift'].mean()  # demean
# design_matrix['v_shift'] -= design_matrix['v_shift'].mean()  # demean
# design_matrix['z_shift'] = design_matrix.groupby('dataset_amsterdam').z_shift.apply(lambda x: (x-x.mean())/x.std())
# design_matrix['v_shift'] = design_matrix.groupby('dataset_amsterdam').v_shift.apply(lambda x: (x-x.mean())/x.std())
print(design_matrix.to_string(index=False))

 intercept  dataset
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1       -1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1
         1        1


In [44]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# import pandas as pd 
# all_parameters = pd.read_pickle('./derivatives/all_subjectwise_parameters.pkl')
# all_parameters = all_parameters[['ddm z_cue_regressor', 'ddm difficulty_effect']]

# all_parameters_dmc = pd.read_csv('./derivatives/BMAParameters.csv')

# f, ax = plt.subplots(1,3)
# ax[0].plot(all_parameters_dmc['vshift'], all_parameters['ddm difficulty_effect'], '.')
# ax[0].set_xlabel('DMC drift shift')
# ax[0].set_ylabel('HDDM drift shift')

# ax[1].plot(all_parameters_dmc['zshiftBias'], all_parameters['ddm z_cue_regressor'], '.')
# ax[1].set_xlabel('DMC z-shift')
# ax[1].set_ylabel('HDDM z-shift')

# ax[2].plot(all_parameters_dmc['vshiftBias'], all_parameters['ddm z_cue_regressor'], '.')
# ax[2].set_xlabel('DMC v-shift (bias)')
# ax[2].set_ylabel('HDDM z-shift')
# f.set_size_inches(15,4)

# # so HDDM and DMC agree on the drift rate shift due to the difficulty effect, 
# # but not on the start point shift due to the cue

In [45]:
# # Let's try the DMC parameters, which are hopefully a bit better
# # The third-level GLM has four EVs (and an intercept)

# design_matrix = pd.DataFrame([1] * all_parameters.shape[0], columns=['intercept'])
# design_matrix['dataset'] = [-1] * 19 + [1] * 14
# design_matrix['z_shift'] = all_parameters_dmc['zshiftBias'].values
# design_matrix['v_shift'] = all_parameters_dmc['vshift'].values
# design_matrix['v_shift_bias'] = all_parameters_dmc['vshiftBias'].values
# design_matrix['z_shift'] -= design_matrix['z_shift'].mean()            # demean
# design_matrix['v_shift'] -= design_matrix['v_shift'].mean()            # demean
# design_matrix['v_shift_bias'] -= design_matrix['v_shift_bias'].mean()  # demean

# print(design_matrix.to_string(index=False))

In [47]:
def fix_fsf_file_flame1(cope,
                        orig_fsf='./derivatives/both/glm_ppi_level3/third_level_design.fsf', 
                        write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    # output directory
    fsf = fsf.replace("/home/stevenm/bias/derivatives/both/glm_ppi_level3/cope1",
                      "/home/stevenm/bias/derivatives/both/glm_ppi_level3/cope{}".format(cope))
    
    for ds in [1,2]:
        for sub in np.arange(1,20):
            fsf = fsf.replace('/home/stevenm/bias/derivatives/ds-0{}/feat_files_ppi/sub-{}/secondlevel.gfeat/cope1.feat'.format(ds, str(sub).zfill(2)),
                              '/home/stevenm/bias/derivatives/ds-0{}/feat_files_ppi/sub-{}/secondlevel.gfeat/cope{}.feat'.format(ds, str(sub).zfill(2), cope))
        
    # save fsf as txt
    if write_out:
        output_dir = '/home/stevenm/bias/derivatives/both/glm_ppi_level3/fsfs/cope{}'.format(cope)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
copes = np.arange(1, 22)

for cope in copes:
    fix_fsf_file_flame1(cope=cope, write_out=True)

In [ ]:
def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/both/glm_ppi_level3/fsfs/cope*/design.fsf'))
all_fsfs

with mp.Pool(15) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/both/glm_ppi_level3/fsfs/cope19/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope21/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope18/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope3/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope17/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope13/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope10/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope11/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope1/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope12/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope20/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope14/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope16/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope15/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope2/design.fsf
./derivatives/both/glm_ppi_level3/fsfs/cope17/design.fsf: CompletedProcess(args=['feat', './derivatives/both/glm_ppi_level3/fsfs/cope17/design.fsf']